In [1]:
import numpy as np
import torch
import ot
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
import sys
sys.path.append('../code/')
sys.path.append('../code/Latent_OT')
from HierarchicalOT import load_wmd_data,change_embeddings,reduce_vocab,fit_topics,sparseOT
from sinkhorn_iterates import sinkhorn
from Latent_OT import LOT_fixsupp
from linear_solver import UOT_W

In [3]:
data_path = './data/WordMoverDistances/'
embeddings_path = './data/WordMoverDistances/glove.6B/glove.6B.300d.txt'

# Pick a dataset (n_doc,n_vocab)
#data_name = 'bbcsport-emd_tr_te_split.mat' #(737,3657)
#data_name = 'twitter-emd_tr_te_split.mat' #(3108, 1205)
#data_name = 'r8-emd_tr_te3.mat' # (7674,5495)
#data_name = 'amazon-emd_tr_te_split.mat' #(8000, 16753)
data_name = 'classic-emd_tr_te_split.mat' # (7093, 5813)
#data_name = 'ohsumed-emd_tr_te_ix.mat'# (9152, 8261)

vocab, embed_vocab, bow_data, y = load_wmd_data(data_path + data_name)
y = y - 1
vocab, embed_vocab, bow_data = change_embeddings(vocab, bow_data, embeddings_path) # embed in Glove

vocab, embed_vocab, bow_data = reduce_vocab(bow_data, vocab, embed_vocab, embed_aggregate='mean') # Reduce vocabulary by removing short words, stop words, and stemming (root words?)
embeddings = np.array([embed_vocab[w] for w in vocab])
cost_embeddings = euclidean_distances(embeddings, embeddings) ** 2 # Matrix of word embeddings: nb_vocab x 300

# WMD Distance

In [ ]:
# UOT penalization
WMD=np.zeros((bow_data.shape[0],bow_data.shape[0]))
for i in range(bow_data.shape[0]):
    print(i,end=" ")
    for j in range(i + 1, bow_data.shape[0]):
        #print(j,end=" ")
        a,b,C=sparseOT(bow_data[i], bow_data[j], cost_embeddings)
        WMD[i,j]=ot.emd2(a,b,C)
WMD = WMD + WMD.T
np.savetxt("result/WordMoverDistances/WMD_"+data_name+".txt",WMD)

# HOTT

In [ ]:
topics, lda_centers, topic_proportions = fit_topics(bow_data, embeddings, vocab, K=70)

# Reduce topics to top-20 words via threshold
# topics is now sparse
n_words_keep = 20
if n_words_keep is not None:
    for k in range(70):
        to_0_idx = np.argsort(-topics[k])[n_words_keep:]
        topics[k][to_0_idx] = 0
        
# Compute WD for the unormalized topics[i],topics[j]. 
# It then defines cost matrix for distrib on topics
cost_topics = np.zeros((topics.shape[0], topics.shape[0]))        
for i in range(cost_topics.shape[0]):
    for j in range(i + 1, cost_topics.shape[1]):
        a,b,C=sparseOT(topics[i], topics[j], cost_embeddings)
        cost_topics[i,j]=ot.emd2(a,b,C)
cost_topics=cost_topics+cost_topics.T #sparse_ot is symmetric

HOTT = np.zeros((bow_data.shape[0], bow_data.shape[0]))        
for i in range(bow_data.shape[0]):
    print(i,end=" ")
    for j in range(i + 1, bow_data.shape[0]):
        a,b,C=sparseOT(topic_proportions[i], topic_proportions[j], cost_topics)
        HOTT[i,j]=ot.emd2(a,b,C)
HOTT= HOTT+HOTT.T
np.savetxt("result/WordMoverDistances/HOTT_"+data_name+".txt",HOTT)

# Latent OT

In [4]:
eps=3

topics, lda_centers, topic_proportions = fit_topics(bow_data, embeddings, vocab, K=2);
n_words_keep = 20
if n_words_keep is not None:
    for k in range(2):
        to_0_idx = np.argsort(-topics[k])[n_words_keep:]
        topics[k][to_0_idx] = 0

LOT1 = np.zeros((bow_data.shape[0], bow_data.shape[0]))        
LOT2 = np.zeros((bow_data.shape[0], bow_data.shape[0]))        
"""for i in range(bow_data.shape[0]):
    print(i,end=" ")
    for j in range(i + 1, bow_data.shape[0]):
        ax,bx,Cx=sparseOT(bow_data[i], topics[0], cost_embeddings)
        az,bz,Cz=sparseOT(topics[0], topics[1], cost_embeddings)
        ay,by,Cy=sparseOT(topics[1], bow_data[j], cost_embeddings)
        Gx = np.exp(-Cx / eps)
        Gz = np.exp(-Cz / eps)
        Gy = np.exp(-Cy / eps)

        Px, Py, Pz, _=LOT_fixsupp(Gx, Gz, Gy,ax,by, niter=20)
        LOT2[i,j]=np.sum(Pz*Cz)+np.sum(Px*Cx)+np.sum(Py*Cy)
        LOT1[i,j]=np.sum(Pz*Cz)

LOT1= LOT1+LOT1.T
LOT2= LOT2+LOT2.T"""
np.savetxt("result/WordMoverDistances/LOTP_"+data_name+".txt",LOT1)
np.savetxt("result/WordMoverDistances/LOT_"+data_name+".txt",LOT2)

INFO:lda:n_documents: 7093
INFO:lda:vocab_size: 5813
INFO:lda:n_words: 371667
INFO:lda:n_topics: 2
INFO:lda:n_iter: 100
INFO:lda:<0> log likelihood: -3056521
INFO:lda:<10> log likelihood: -2794782
INFO:lda:<20> log likelihood: -2718899
INFO:lda:<30> log likelihood: -2704566
INFO:lda:<40> log likelihood: -2696540
INFO:lda:<50> log likelihood: -2692541
INFO:lda:<60> log likelihood: -2690689
INFO:lda:<70> log likelihood: -2690107
INFO:lda:<80> log likelihood: -2689343
INFO:lda:<90> log likelihood: -2689083
INFO:lda:<99> log likelihood: -2688697


In [11]:
topics[0]

array([9.21785160e-03, 1.83444373e-02, 9.21785160e-03, ...,
       9.12658574e-05, 9.12658574e-05, 9.12658574e-05])

In [9]:
eps = 3
Cx,Cy,Cz=ot.dist(X,Zx),ot.dist(Zy,Y),ot.dist(Zx,Zy)
Gx = np.exp(-Cx / eps)
Gz = np.exp(-Cz / eps)
Gy = np.exp(-Cy / eps)

Px, Py, Pz, P=LOT_fixsupp(Gx, Gz, Gy,a,b, niter=20)
print(np.sum(Px*Cx)+np.sum(Py*Cy)+np.sum(Pz*Cz))

array([[9.21785160e-03, 1.83444373e-02, 9.21785160e-03, ...,
        9.12658574e-05, 9.12658574e-05, 9.12658574e-05],
       [8.50557115e-05, 8.50557115e-05, 2.56017692e-02, ...,
        8.50557115e-05, 8.50557115e-05, 8.50557115e-05]])

In [ ]:

n_words_keep = 20
if n_words_keep is not None:
    for k in range(2):
        to_0_idx = np.argsort(-topics[k])[n_words_keep:]
        topics[k][to_0_idx] = 0

topics

In [ ]:
# Reduce topics to top-20 words via threshold
# topics is now sparse
n_words_keep = 20
if n_words_keep is not None:
    for k in range(70):
        to_0_idx = np.argsort(-topics[k])[n_words_keep:]
        topics[k][to_0_idx] = 0
        
# Compute WD for the unormalized topics[i],topics[j]. 
# It then defines cost matrix for distrib on topics
cost_topics = np.zeros((topics.shape[0], topics.shape[0]))        
for i in range(cost_topics.shape[0]):
    for j in range(i + 1, cost_topics.shape[1]):
        a,b,C=sparseOT(bow_data[i], bow_data[j], cost_embeddings)
        cost_topics[i,j]=ot.emd2(a,b,C)
cost_topics=cost_topics+cost_topics.T #sparse_ot is symmetric

HOTT = np.zeros((bow_data.shape[0], bow_data.shape[0]))        
for i in range(bow_data.shape[0]):
    print(i,end=" ")
    for j in range(i + 1, bow_data.shape[0]):
        a,b,C=sparseOT(topic_proportions[i], topic_proportions[j], cost_topics)
        HOTT[i,j]=ot.emd2(a,b,C)
HOTT= HOTT+HOTT.T
#np.savetxt("result/WordMoverDistances/HOTT_"+data_name+".txt",HOTT)

# HROT

In [ ]:
# Linear Solver
lam=[.1,.5,1,10]
for l in lam:
    WMDuot1=np.zeros((bow_data.shape[0],bow_data.shape[0]))
    WMDuot2=np.zeros((bow_data.shape[0],bow_data.shape[0]))
    #for i in range(bow_data.shape[0]):
    #    print(i,end= " ")
    #    for j in range(i + 1, bow_data.shape[0]):
    #        #print(j,end=" ")
    #        a,b,C=sparseOT(bow_data[i], bow_data[j], cost_embeddings)
    #        _,_,Cx=sparseOT(bow_data[i], bow_data[i], cost_embeddings)
    #        _,_,Cy=sparseOT(bow_data[j], bow_data[j], cost_embeddings)
    #        P,Qx,Qy=UOT_W(a,b,C,lam=l,Cx=Cx,Cy=Cy,innerplan=True,solver="CLARABEL")
    #        WMDuot1[i,j]=np.sum(np.multiply(P,C))
    #        WMDuot2[i,j]=np.sum(np.multiply(P,C))+l*(np.sum(Qx*Cx)+np.sum(Qy*Cy))
    #WMDuot1 = WMDuot1 + WMDuot1.T
    #WMDuot2 = WMDuot2 + WMDuot2.T
    #np.savetxt("result/WordMoverDistances/UOTP_"+data_name+"_"+str(l)+".txt",WMDuot1) #Only with the plan
    #np.savetxt("result/WordMoverDistances/UOT_"+data_name+"_"+str(l)+".txt",WMDuot2) #Plan and divergences

In [ ]:
# Sinkhorn Algorithm
lam=[.1]#[.1,.5,1,10]
for l in lam:
    WMDuot1=np.zeros((bow_data.shape[0],bow_data.shape[0]))
    WMDuot2=np.zeros((bow_data.shape[0],bow_data.shape[0]))
    for i in range(bow_data.shape[0]):
        print(i, end=" ")
        for j in range(i + 1, bow_data.shape[0]):
            #print(j,end=" ")
            a,b,C=sparseOT(bow_data[i], bow_data[j], cost_embeddings)
            _,_,Cx=sparseOT(bow_data[i], bow_data[i], cost_embeddings)
            _,_,Cy=sparseOT(bow_data[j], bow_data[j], cost_embeddings)
            if (Cx.shape[0] == a.shape[0]) and (Cy.shape[0] == b.shape[0]):
                P,Qx,Qy=sinkhorn(torch.tensor(a),torch.tensor(b),torch.tensor(C),lam=l,
                             eps=3,numiter=20,lam2=None,pen="sinkhorn",Cx=torch.tensor(Cx),
                             Cy=torch.tensor(Cy),numiter2=1,innerplan=True)
                P,Qx,Qy=P.numpy(),Qx.numpy(),Qy.numpy()
                WMDuot1[i,j]=np.sum(np.multiply(P,C))
                WMDuot2[i,j]=np.sum(np.multiply(P,C))+l*(np.sum(Qx*Cx)+np.sum(Qy*Cy))
    WMDuot1 = WMDuot1 + WMDuot1.T
    WMDuot2 = WMDuot2 + WMDuot2.T
    #np.savetxt("result/WordMoverDistances/UOTeP_"+data_name+"_"+str(10)+".txt",WMDuot1) #Only with the plan
    #np.savetxt("result/WordMoverDistances/UOTe_"+data_name+"_"+str(10)+".txt",WMDuot2) #Plan and divergences

In [24]:
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [26]:
x=5
print('ui')
with suppress_stdout():
    x+=5
    print("Now you don't")

ui


In [27]:
x

10